In [1]:
#imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import sys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import pandas as pd
import csv
import re
import os
from datetime import datetime

In [2]:
# Get user input for filename prefix
filename_prefix = input("Enter your file name followed by the current semester: ")
date_str = datetime.today().strftime('%Y-%m-%d')

# Create directories if they don't exist
os.makedirs("webscrapedInfo", exist_ok=True)
os.makedirs("intermediateFiles", exist_ok=True)
os.makedirs("finalFiles", exist_ok=True)

# Define filenames with respective directories
output_filename = f"webscrapedInfo/{filename_prefix}_Output_{date_str}.csv"
all_faculty_info_filename = f"webscrapedInfo/{filename_prefix}_AllFacultyInfo_{date_str}.csv"
exclusive_hours_filename = f"intermediateFiles/{filename_prefix}_Exclusive_Hours_{date_str}.csv"
exclusive_hours_cleaned_filename = f"intermediateFiles/{filename_prefix}_Exclusive_Hours_Cleaned_{date_str}.csv"
formatted_instructor_filename = f"intermediateFiles/{filename_prefix}_Formatted_Instructor_{date_str}.csv"
formatted_instructor_military_filename = f"intermediateFiles/{filename_prefix}_Formatted_Instructor_Military_{date_str}.csv"
formatted_instructor_further_filename = f"intermediateFiles/{filename_prefix}_Formatted_Instructor_FurtherTimeFormat_{date_str}.csv"
formatted_instructor_further_updated_filename = f"intermediateFiles/{filename_prefix}_Formatted_Instructor_FurtherTimeFormat_Updated_{date_str}.csv"
formatted_instructor_expanded_filename = f"intermediateFiles/{filename_prefix}_Formatted_Instructor_FurtherTimeFormat_Expanded_{date_str}.csv"
formatted_instructor_expanded_lowercase_filename = f"intermediateFiles/{filename_prefix}_Formatted_Instructor_FurtherTimeFormat_Expanded_Lowercase_{date_str}.csv"
formatted_instructor_expanded_lowercase_updated_filename = f"intermediateFiles/{filename_prefix}_Formatted_Instructor_FurtherTimeFormat_Expanded_Lowercase_Updated_{date_str}.csv"
final_filename = f"finalFiles/{filename_prefix}_Formatted_Instructor_FurtherTimeFormat_Expanded_Lowercase_Updated_Office_Changes_{date_str}.csv"

In [3]:
#Driver for firefox and loading the Eweb url
driver = webdriver.Firefox()
driver.implicitly_wait(10)

driver.get("https://www.easternct.edu/faculty-directory/index.html")

In [4]:
#selecting SearchButton
Search = driver.find_element(by = 'id', value = 'directorySearchButton')
Search.click()

In [5]:
#Locating the HTML where the info is stored
table = driver.find_elements(by = 'class name', value = "mix")

In [6]:
print(len(table))

872


In [7]:
for t in table[1:]:
    t.text
# Assuming table is already defined
data = [t.text.strip() for t in table[1:]]

# Create a DataFrame
df = pd.DataFrame(data, columns=["Extracted Text"])

# Save to CSV
df.to_csv(output_filename, index=False)

print(f"CSV file saved as '{output_filename}'")

CSV file saved as 'webscrapedInfo/6TestSpring2025_Output_2025-05-07.csv'


In [8]:
#Close the webpage
driver.quit()

In [9]:
def parse_professor_info(text):
    """
    Parse professor info from a multi-line string into a dictionary.
    Expected format:
      Line 1: Name
      Line 2: Position
      Line 3: Department
      Subsequent lines: 'Phone:', 'Email:', 'Office:' and optionally 'Hours:'
    """
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    professor = {
        'name': None,
        'position': None,
        'department': None,
        'phone': None,
        'email': None,
        'office': None,
        'Hours': None
    }
    
    if len(lines) >= 1:
        professor['name'] = lines[0]
    if len(lines) >= 2:
        professor['position'] = lines[1]
    if len(lines) >= 3:
        professor['department'] = lines[2]
    
    for line in lines:
        if line.startswith('Phone:'):
            professor['phone'] = line.replace('Phone:', '').strip()
        elif line.startswith('Email:'):
            professor['email'] = line.replace('Email:', '').strip()
        elif line.startswith('Office:'):
            professor['office'] = line.replace('Office:', '').strip()
        elif line.startswith('Hours:'):
            professor['Hours'] = line.replace('Hours:', '').strip()
    
    return professor

def main():
    # Load the CSV file with extracted text
    df = pd.read_csv(output_filename)
    
    # Parse each row and create a new DataFrame with structured data
    parsed_data = df["Extracted Text"].apply(parse_professor_info).tolist()
    parsed_df = pd.DataFrame(parsed_data)
    
    # Save the structured DataFrame to a new CSV file
    parsed_df.to_csv(all_faculty_info_filename, index=False)
    print(f"Parsed CSV saved as '{all_faculty_info_filename}'")

if __name__ == '__main__':
    main()

Parsed CSV saved as 'webscrapedInfo/6TestSpring2025_AllFacultyInfo_2025-05-07.csv'


In [10]:
# Filtering 'Hours' field
df = pd.read_csv(all_faculty_info_filename)
df_filtered = df[df['Hours'].notna() & (df['Hours'].str.strip() != '')]
df_filtered.to_csv(exclusive_hours_filename, index=False)
print(f"Filtered data saved as '{exclusive_hours_filename}'")

Filtered data saved as 'intermediateFiles/6TestSpring2025_Exclusive_Hours_2025-05-07.csv'


In [11]:
#Fixing Jr. name error
# Load the CSV file
df = pd.read_csv(exclusive_hours_filename)

# Remove 'Jr.' if it appears at the end of a name with preceding spaces
# \s+  -> Matches one or more spaces before "Jr."
# Jr\. -> Matches "Jr."
# \s*$ -> Matches any trailing spaces at the end
df["name"] = df["name"].str.replace(r"\s+Jr\.\s*$", "", regex=True).str.strip()

# Save the modified CSV
df.to_csv(exclusive_hours_filename, index=False)

print(f"Filtered data saved as '{exclusive_hours_filename}'")

Filtered data saved as 'intermediateFiles/6TestSpring2025_Exclusive_Hours_2025-05-07.csv'


In [12]:
#name formatting
# Load the CSV file
df = pd.read_csv(exclusive_hours_filename)

# Remove 'Jr.' if it appears at the end of a name
df["name"] = df["name"].str.replace(r"\s+Jr\.\s*$", "", regex=True).str.strip()

# Format the name as "Last, First Initial"
def format_name(name):
    name_parts = name.split()
    if len(name_parts) >= 2:
        last_name = name_parts[-1]  # Last name is the last word
        first_initial = name_parts[0][0]  # First initial from first word
        return f"{last_name}, {first_initial}"
    return name  # If format is unexpected, keep the name as is

df["name"] = df["name"].apply(format_name)

# Save the cleaned CSV
df.to_csv(exclusive_hours_cleaned_filename, index=False)

print(f"Updated file saved as '{exclusive_hours_cleaned_filename}'")

Updated file saved as 'intermediateFiles/6TestSpring2025_Exclusive_Hours_Cleaned_2025-05-07.csv'


In [13]:
#Shrinking to desired rows
# Load the cleaned CSV
df = pd.read_csv(exclusive_hours_cleaned_filename)

# Select only the required columns
df_filtered = df[["name", "department", "office", "Hours"]]

# Save the formatted CSV
df_filtered.to_csv(formatted_instructor_filename, index=False)

print(f"Formatted file saved as '{formatted_instructor_filename}'")

Formatted file saved as 'intermediateFiles/6TestSpring2025_Formatted_Instructor_2025-05-07.csv'


In [14]:
#TimeFormatting
# Load the CSV
df = pd.read_csv(formatted_instructor_filename)

# Function to convert time to military format
def convert_to_military(time_str):
    if not isinstance(time_str, str) or "sabbatical" in time_str.lower():
        return time_str  # Keep non-time values unchanged

    # Normalize AM/PM format (remove dots and spaces)
    time_str = re.sub(r"\s*(a\.?m\.?|p\.?m\.?)", lambda m: m.group(1).replace(".", "").upper(), time_str)

    # Regular expression to match time formats like '3:00PM', '8AM-3PM', '12:00-1 PM'
    time_pattern = re.compile(r"(\d{1,2}:\d{2}|\d{1,2})\s*(AM|PM)?")

    def convert_match(match):
        """Convert matched time to 24-hour format while handling missing AM/PM properly."""
        time_text, am_pm = match.groups()
        
        # If AM/PM is missing but part of a range, infer from previous time
        if not am_pm:
            return time_text  # Keep as-is if AM/PM context is unclear

        # Convert to 24-hour format
        time_obj = datetime.strptime(f"{time_text}{am_pm}", "%I:%M%p" if ":" in time_text else "%I%p")
        return time_obj.strftime("%H:%M")

    # Replace all matched times in the string
    time_str = time_pattern.sub(convert_match, time_str)

    # Remove any leftover "AM" or "PM" that might still be in the string
    time_str = re.sub(r"\s*(AM|PM)\b", "", time_str)

    return time_str.strip()  # Ensure clean formatting

# Apply conversion to the "Hours" column
df["Hours"] = df["Hours"].apply(convert_to_military)

# Save the updated CSV
df.to_csv(formatted_instructor_military_filename, index=False)

print(f"Updated file saved as '{formatted_instructor_military_filename}'")


Updated file saved as 'intermediateFiles/6TestSpring2025_Formatted_Instructor_Military_2025-05-07.csv'


In [15]:
# Mapping of day name patterns to condensed formats
# Load the CSV
df = pd.read_csv(formatted_instructor_military_filename)
# === Mapping of day name patterns to condensed formats ===
day_mapping = {
    "M, W, F": "MWF", "M, W": "MW", "M, F": "MF", "M, Tu": "MT",
    "Tu, Th": "TR", "M-W": "MW", "M-F": "MTWRF", "W, F": "WF",
    "M": "M", "Tu": "T", "W": "W", "Th": "R", "F": "F",
    "M, Tu, W": "MTW", "M, W, Th": "MWR", "Tu, W": "TW",
    "Th, F": "RF", "M, Th": "MT", "Tu, W, Th": "TWR",
    "M, W, F, Th": "MWRF", "M, F, Th": "MFT", "T, R, F": "TRF"
}

# === Converts time to military format ===
def format_time(time_str):
    if time_str.strip() == "1":
        return "1300"
    time_str = time_str.replace(":", "")
    if re.match(r"^\d{3}$", time_str):
        time_str = f"0{time_str}"
    if time_str.isdigit():
        time_int = int(time_str)
        if time_int < 630:
            time_int += 1200
        return f"{time_int:04d}"
    return time_str

# === Expands 'and' in each time block ===
def expand_and_cases(s):
    new_parts = []

    # Split multiple entries by semicolon
    for part in s.split(";"):
        part = part.strip()
        match = re.match(r"([A-Za-z ,]+):\s*(.*)", part)
        if not match:
            new_parts.append(part)
            continue

        days_raw, times_raw = match.groups()
        days = [d.strip() for d in days_raw.split(",")]
        time_ranges = [t.strip() for t in re.split(r"\band\b", times_raw)]

        for day in days:
            for tr in time_ranges:
                new_parts.append(f"{day}: {tr}")

    return "; ".join(new_parts)

# === Formats full string of hours ===
def format_hours(hours_str):
    if not isinstance(hours_str, str) or "sabbatical" in hours_str.lower():
        return hours_str

    hours_str = expand_and_cases(hours_str)

    # Replace verbose day names with abbreviations
    for full_days, short_days in day_mapping.items():
        hours_str = re.sub(rf"\b{re.escape(full_days)}\b", short_days, hours_str)

    # Extract and format time blocks
    pattern = re.compile(r"([MTWRF]):\s*([\d:]+)-([\d:]+)")
    formatted_entries = []
    for match in pattern.finditer(hours_str):
        day, start_time, end_time = match.groups()
        formatted_entries.append(f"{day}, {format_time(start_time)}, {format_time(end_time)}")

    return ", ".join(formatted_entries)

# === Load and process ===
df = pd.read_csv(formatted_instructor_military_filename)
df["Hours"] = df["Hours"].apply(format_hours)
df.to_csv(formatted_instructor_further_filename, index=False)

print(f" Updated file saved as: {formatted_instructor_further_filename}")

 Updated file saved as: intermediateFiles/6TestSpring2025_Formatted_Instructor_FurtherTimeFormat_2025-05-07.csv


In [16]:
#Fix names
# Load the CSV file
df = pd.read_csv(formatted_instructor_further_filename)

# Assuming the column with names is called 'name', adjust if necessary
df['name'] = df['name'].str.replace(',', '').str.replace('-', '').str.replace(' ', '').str.replace("'", '')

# Save the updated dataframe back to a new CSV
df.to_csv(formatted_instructor_further_updated_filename, index=False)

In [17]:
#Room format
# Load the CSV file
df = pd.read_csv(formatted_instructor_further_updated_filename)

# Create a dictionary for the replacements
replacement_dict = {
    'Goddard Hall': 'goddard',
    'Communication': 'communications',
    'Webb Hall': 'webb',
    'Center for Early Childhood Education': 'the_center_for_early_childhood_education',
    'Fine Arts': 'fine_arts',
    'Gelsi Young': 'gelsi,'
}

# Replace using the dictionary
df['office'] = df['office'].replace(replacement_dict, regex=True)

# Save the updated dataframe back to a new CSV
df.to_csv(formatted_instructor_further_updated_filename, index=False)

In [18]:
#Splitting office hour times
# Load the CSV file
df = pd.read_csv(formatted_instructor_further_updated_filename)

# Function to expand hours into separate rows, splitting every third comma
def expand_hours(row):
    # Skip non-standard values like 'On Sabbatical Leave'
    if isinstance(row['Hours'], str) and row['Hours'] not in ['On Sabbatical Leave', 'On Sabbatical S25']:
        hours_list = row['Hours'].split(', ')  # Split by commas
        new_rows = []

        # Ensure we have a valid number of elements, every 3rd comma should form a new entry
        if len(hours_list) % 3 != 0:
            print(f"Skipping invalid format in row with Hours: {row['Hours']}")
            return []  # Skip this row if the format is invalid

        # Create a new row for each time slot (every 3rd element is the time slot)
        for i in range(0, len(hours_list), 3):
            new_row = row.copy()
            new_row['Hours'] = f"{hours_list[i]}, {hours_list[i+1]}, {hours_list[i+2]}"  # Form day, time, time slot
            new_rows.append(new_row)

        return new_rows
    else:
        # If 'Hours' is not a valid string or contains sabbatical info, return the row as is
        return [row]

# Create a new list to hold expanded rows
expanded_rows = []

# Apply the function to each row and expand hours
for _, row in df.iterrows():
    expanded_rows.extend(expand_hours(row))  # Expand hours for the row

# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)

# Save the updated dataframe back to a new CSV
expanded_df.to_csv(formatted_instructor_expanded_filename, index=False)





Skipping invalid format in row with Hours: (Spring 2025) On Sabbatical Leave- Contact Department Secretary
Skipping invalid format in row with Hours: On Sabbatical Leave for F24/S25


In [19]:
#LowerCase

# Load the CSV file
df = pd.read_csv(formatted_instructor_expanded_filename)

# Convert all columns to lowercase using apply() with axis=0 to apply to each column
df = df.apply(lambda col: col.str.lower() if col.dtype == "object" else col)

# Save the updated dataframe back to a new CSV
df.to_csv(formatted_instructor_expanded_lowercase_filename, index=False)

In [20]:
#Underscore instead of spaces department and removing " "
# Load the CSV file
df = pd.read_csv(formatted_instructor_expanded_lowercase_filename)

# Replace spaces with underscores in the 'department' column
df['department'] = df['department'].str.replace(' ', '_')

# Save the updated dataframe back to a new CSV without extra quotes
df.to_csv(formatted_instructor_expanded_lowercase_updated_filename, index=False)

In [21]:
#Room Touch up removing the or in greenhouse and science
# Load the CSV file
df = pd.read_csv(formatted_instructor_expanded_lowercase_updated_filename)

# Define a function to handle the replacements in the 'office' column
def adjust_office(office):
    # Ensure "Science" is followed by a comma
    if "science" in office.lower() and not office.lower().startswith("science,"):
        office = office.replace("science", "science,")
    
    # Replace " or " with a comma
    office = office.replace(" or ", ", ")
    
    # Add a comma after "greenhouse" if not already present
    if "greenhouse" in office.lower() and not office.endswith("greenhouse,") and "greenhouse" in office:
        office = office.replace("greenhouse", "greenhouse,")
    
    return office

# Apply the function to the 'office' column
df['office'] = df['office'].apply(adjust_office)

df['office'] = df['office'].str.replace(', ', '_').str.replace(' ', '_')
df.loc[df['Hours'].str.contains('sab', case=False, na=False), 'Hours'] = 'null, null, null'

df['department'] = df['department'].str.replace(',', '')
df['department'] = df['department'] + ', '
df['Hours'] = ' '+ df['Hours']
df['office'] = df['office'].str.replace("_room","")

# Save the updated dataframe back to a new CSV
df.to_csv(final_filename, index=False)


In [22]:
#greenhouse
# Load the CSV file
df = pd.read_csv(final_filename)

# Create an empty list to store the updated rows
updated_rows = []

# Iterate over each row in the dataframe
for _, row in df.iterrows():
    office = row['office']
    
    # Check if 'greenhouse room' is in the office field
    if 'greenhouse room' in office:
        # Create the first row (original) - split the office at 'greenhouse room' and keep the first part
        new_row_1 = row.copy()
        office_parts = office.split('greenhouse room', 1)  # Split at the first occurrence of 'greenhouse room'
        new_row_1['office'] = office_parts[0]  # Keep only the part before 'greenhouse room'
        updated_rows.append(new_row_1)  # Add the original (modified) row
        
        # Create the second row (duplicate) - keep the part after 'greenhouse room'
        new_row_2 = row.copy()
        new_row_2['office'] = 'greenhouse room' + office_parts[1]  # Keep only the part after 'greenhouse room'
        updated_rows.append(new_row_2)  # Add the duplicate (modified) row
    else:
        # If 'greenhouse room' isn't found, just add the current row
        updated_rows.append(row)

# Create a new DataFrame from the updated rows
updated_df = pd.DataFrame(updated_rows)

#replacing the slash with underscore
updated_df['office'] = updated_df['office'].str.replace('/', '_')


# Save the updated dataframe back to a new CSV file
updated_df.to_csv(final_filename, index=False)

In [23]:
#Combining into one string
# Load the final instructor course schedule CSV
df = pd.read_csv(final_filename)

df['office'] = df['office'].str.replace(r'_(?=[^_]*$)', ', ', regex=True)
df['office'] = df['office'].str.replace(r'(\d+)[a-zA-Z]$', r'\1', regex=True)


# Function to combine the columns into a single string for each row with a space after each comma
def combine_columns(row):
    return f"{row['name']}, {row['department']}{row['office']},{row['Hours']}"

# Apply the function to combine the columns for each row into one formatted string
df_combined = df.apply(combine_columns, axis=1)

# Create a new DataFrame with the combined strings
df_prolog_ready = pd.DataFrame(df_combined, columns=["officeHours"])

df_prolog_ready["officeHours"] = df_prolog_ready["officeHours"].str.replace(r'nan$', 'null, null, null', regex=True)

# Sort the DataFrame alphabetically based on the first character of the 'Courses' column
df_prolog_ready = df_prolog_ready.sort_values(by="officeHours", ascending=True).reset_index(drop=True)

# Export the combined and sorted DataFrame to a new CSV
df_prolog_ready.to_csv(final_filename, index=False)

print(f"Exported to {final_filename}")

Exported to finalFiles/6TestSpring2025_Formatted_Instructor_FurtherTimeFormat_Expanded_Lowercase_Updated_Office_Changes_2025-05-07.csv


#Adding in the department and formatting it : Done
Format like prolog : done 
Convert time to military time and format it : Done
Convert the days to the same representation : Done

